In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler

In [ ]:
"""Function that prepares the input X and output y for training a LSTM network


Args:
    sequence_1: the first sequence which gets converted into multiple subarrays of length: n_steps
    sequence_2: the second sequence, each n_steps'th element will be part of the output array
    n_steps: the amount of time steps used as an input into the LSTM for prediction

Returns:
    A tuple of 2 numpy arrays in the required format
    
    X.shape = (X.shape[0] - n_steps, n_steps)
    y.shape = (X.shape[0] - n_steps)

"""
def prepare_sequences(sequence_1, sequence_2, n_steps):
    X, y = list(), list()
    for i in range(len(sequence_1)):
        end_ix = i + n_steps

        if end_ix > len(sequence_1)-1:
            break

        seq_x = sequence_1[i:end_ix]
        X.append(seq_x)
        y.append(sequence_2[end_ix])
        
    return np.array(X), np.array(y)


def subsample(sequence, d_t):

    subsampled_sequence = np.array([])

    for i in range(0, len(sequence), d_t - 1) :
        
        if len(sequence) < i + d_t:
            # check if at end of sequence
            window = sequence[i:len(sequence)]
        else:
            window = sequence[i:i + d_t]
        
        # search for most frequent element in observed window
        uniqw, inverse = np.unique(window, return_inverse=True)
        frequent_element_index = np.bincount(inverse).argmax()
        frequent_element = window[frequent_element_index]
    
        # add most frequent element to subsample
        subsampled_sequence = np.append(subsampled_sequence, frequent_element)
        
    return subsampled_sequence

def smoothing(sequence, d_t):

    subsampled_sequence = np.array([])

    for i in range(0, len(sequence), d_t - 1) :
        
        if len(sequence) < i + d_t:
            # check if at end of sequence
            window = sequence[i:len(sequence)]
            seq = np.ones((len(sequence) - i, 1))
        else:
            window = sequence[i:i + d_t]
            seq = np.ones((d_t, 1))
        
        # search for most frequent element in observed window
        uniqw, inverse = np.unique(window, return_inverse=True)
        frequent_element_index = np.bincount(inverse).argmax()
        frequent_element = window[frequent_element_index]
    
        # add most frequent element to subsample
        subsampled_sequence = np.append(subsampled_sequence, seq * frequent_element)
        
    return subsampled_sequence

In [ ]:
# set hyperparameters
n_steps = 100
n_features = 1
n_units = 5
n_epochs = 10

In [ ]:
# ------------------------------------------- Prepare Training Data -------------------------------------------
train_cur_master = np.loadtxt('../data/fobss_data/data/Ri Jumps 25A/battery/Battery_Current.csv', delimiter=';')
train_cur_master = train_cur_master[10000:150000,1]
train_cur_master = subsample(train_cur_master, 10)

train_volt_slave_0_cell_4 = np.loadtxt('../data/fobss_data/data/Ri Jumps 25A/cells/Slave_0_Cell_Voltages.csv', delimiter=';')
train_volt_slave_0_cell_4 = train_volt_slave_0_cell_4[10000:150000,4]
train_volt_slave_0_cell_4 = subsample(train_volt_slave_0_cell_4, 10)

# convert into X and y sequences
X_train, y_train = prepare_sequences(train_cur_master, train_volt_slave_0_cell_4, n_steps)

# scale input features
scaler = MinMaxScaler(feature_range = (0, 1))
X_train_scaled = scaler.fit_transform(X_train)

# reshape into correct input format
X_train_scaled = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], n_features)

print(X_train_scaled.shape, y_train.shape)

In [ ]:
# ------------------------------------------- Initialize LSTM -------------------------------------------
model = keras.Sequential()

# Adding the first LSTM layer and some Dropout regularisation
model.add(layers.LSTM(units = n_units, activation='relu', input_shape = (n_steps, n_features)))

# Adding the output layer
model.add(layers.Dense(units = 1))

# Show model
model.summary()

In [ ]:
# ------------------------------------------- Train LSTM -------------------------------------------
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
model.fit(X_train_scaled, y_train, epochs = n_epochs, verbose = 1)

In [ ]:
# ------------------------------------------- Prepare Test Data -------------------------------------------
test_cur_master = np.loadtxt('../data/fobss_data/data/Ri Jumps 25A/battery/Battery_Current.csv', delimiter=';')
test_cur_master = test_cur_master[80000:100000,1]
test_volt_slave_0_cell_4 = np.loadtxt('../data/fobss_data/data/Ri Jumps 25A/cells/Slave_0_Cell_Voltages.csv', delimiter=';')
test_volt_slave_0_cell_4 = test_volt_slave_0_cell_4[80000:100000, 4]

# convert into X and y sequences
X_test, y_test = prepare_sequences(test_cur_master, test_volt_slave_0_cell_4, n_steps)

# scale input features
scaler = MinMaxScaler(feature_range = (0, 1))
X_test_scaled = scaler.fit_transform(X_test)

# reshape into correct input format
X_test_scaled = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], n_features)

print(X_test.shape, y_test.shape)

In [ ]:
# predict on test data
yhat = model.predict(X_test_scaled, verbose = 1)

# plot test results
plt.plot(yhat, color='red', label = 'predicted')
plt.plot(y_test, color='blue', label = 'measured')
plt.legend()
plt.show()